In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from statsmodels.formula.api import ols
from math import exp
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import pickle
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('data/kc_house_data_test_features.csv', index_col=0)

In [3]:
infile = open('model.pickle', 'rb')
final_model = pickle.load(infile)
infile.close()

print(final_model.intercept_)
print(len(final_model.coef_))

9.216524831707037
141


In [4]:
infile = open('columns.pickle', 'rb')
columns = pickle.load(infile)
infile.close()

len(columns)

141

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [5]:
df.reset_index(inplace=True)

In [6]:
df

,index,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4318,4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4319,4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4320,4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [7]:
df.date = pd.to_datetime(df.date)

In [8]:
df['second_sale'] = [1 if x == True else 0 for x in df.duplicated(subset='id', keep='last')]

In [9]:
df = df.sort_values('date').drop(['index','id'], axis=1)

In [10]:
df['bathrooms'] = [1 if x <= 1.25 else (4 if x >=4 else x) for x in df.bathrooms]

In [11]:
df['bedrooms'] = [6 if x >=6 else x for x in df.bedrooms]

In [12]:
df['is_renovated'] = [0 if x == 0 else 1 for x in df.yr_renovated]

In [13]:
df['house_age'] = [2015 - x for x in df.yr_built]

In [14]:
df['has_basement'] = [0 if x == 0 else 1 for x in df.sqft_basement]

In [15]:
df['date'] = [x.strftime('%Y-%m') for x in df.date]

In [16]:
df['date'] = pd.to_datetime(df.date)

In [17]:
df['season'] = ['Winter' if int(x.strftime('%m')) in list([1,2,12]) 
                else ('Spring' if int(x.strftime('%m')) in range(3,6)
                else ('Summer' if int(x.strftime('%m')) in range(6,9)
                else 'Fall')) for x in df.date]


In [18]:
df['yard_space'] = df.sqft_lot - (df.sqft_living / df.floors)
df['yard_space'] = [0 if x < 0 else x for x in df.yard_space]

In [19]:
df['grade'] = [5 if x <= 5 else (11 if x >=11 else x) for x in df.grade]

In [20]:
df['sqft_per_floor'] = df['sqft_living']/df['floors']
df['floor_area_lot_ratio'] = df['sqft_per_floor']/df['sqft_lot']
df['living_bathroom_ratio'] = df.sqft_living/df.bathrooms
df['living_bedroom_ratio'] = df.sqft_living/df.bedrooms

In [21]:
df.replace(np.inf, 0, inplace=True)

In [22]:
df['sqft_living_log'] = [np.log(x) for x in df.sqft_living]

In [23]:
df['floors'] = [3 if x >= 3 else x for x in df.floors]

In [24]:
df['condition'] = [2 if x <= 2 else x for x in df.condition]

In [25]:
df['built_by_decade'] = [round(x,-1) for x in df.yr_built]

In [26]:
df['built_by_decade'] = [2010 if x >= 2020 else x for x in df.built_by_decade]

In [27]:
df['years_from_1970'] = [abs(x-1970) for x in df.built_by_decade]

In [28]:
df = pd.get_dummies(df, columns =['zipcode', 'grade', 'years_from_1970', 'bedrooms', 'bathrooms', 'date', 'floors', 'waterfront', 'view', 'condition', 'built_by_decade','season'], drop_first=True)

In [29]:
df.drop(['zipcode_98002', 'zipcode_98003', 'zipcode_98030', 'zipcode_98031','zipcode_98032',  'zipcode_98042', 'zipcode_98092', 'zipcode_98188', 'zipcode_98198'], axis=1, inplace=True)

In [30]:
df

,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,second_sale,is_renovated,house_age,has_basement,yard_space,sqft_per_floor,floor_area_lot_ratio,living_bathroom_ratio,living_bedroom_ratio,sqft_living_log,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98199,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,years_from_1970_10,years_from_1970_20,years_from_1970_30,years_from_1970_40,years_from_1970_50,years_from_1970_60,years_from_1970_70,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bathrooms_1.5,bathrooms_1.75,bathrooms_2.0,bathrooms_2.25,bathrooms_2.5,bathrooms_2.75,bathrooms_3.0,bathrooms_3.25,bathrooms_3.5,bathrooms_3.75,bathrooms_4.0,date_2014-06-01 00:00:00,date_2014-07-01 00:00:00,date_2014-08-01 00:00:00,date_2014-09-01 00:00:00,date_2014-10-01 00:00:00,date_2014-11-01 00:00:00,date_2014-12-01 00:00:00,date_2015-01-01 00:00:00,date_2015-02-01 00:00:00,date_2015-03-01 00:00:00,date_2015-04-01 00:00:00,date_2015-05-01 00:00:00,floors_1.5,floors_2.0,floors_2.5,floors_3.0,waterfront_1,view_1,view_2,view_3,view_4,condition_3,condition_4,condition_5,built_by_decade_1910,built_by_decade_1920,built_by_decade_1930,built_by_decade_1940,built_by_decade_1950,built_by_decade_1960,built_by_decade_1970,built_by_decade_1980,built_by_decade_1990,built_by_decade_2000,built_by_decade_2010,season_Spring,season_Summer,season_Winter
109,2910,35200,2910,0,1979,0,47.5747,-122.035,2590,37500,0,0,36,0,33260.000000,1940.000000,0.055114,1662.857143,970.000000,7.975908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2539,1770,1235,1600,170,2007,0,47.6965,-122.342,1680,1203,0,0,8,1,645.000000,590.000000,0.477733,708.000000,590.000000,7.478735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1775,1920,10000,1070,850,1954,0,47.5725,-122.133,1450,10836,0,0,61,1,8080.000000,1920.000000,0.192000,1280.000000,480.000000,7.560080,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2858,1320,1327,1040,280,2008,0,47.6506,-122.383,1440,1263,0,0,7,1,667.000000,660.000000,0.497362,406.153846,440.000000,7.185387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
111,2710,4500,1880,830,1929,0,47.6747,-122.295,2060,4500,0,0,86,1,2693.333333,1806.666667,0.401481,1355.000000,903.333333,7.904704,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Step 3: Predict the holdout set

In [31]:
final_answers = final_model.predict(df[columns])
final_answers = np.exp(final_answers)

In [32]:
final_answers = pd.DataFrame(final_answers)

In [33]:
final_answers

,0
0,628596.456476
1,528748.537517
2,496068.251429
3,510075.290920
4,739533.513676
...,...
4317,483215.906982
4318,733063.624039
4319,582568.931102
4320,741773.657672


## Step 4: Export your predictions

In [34]:
final_answers.to_csv('housing_preds_Edgar_Gonzalez.csv')